# AniMate Ver. 1.0.0

This notebook is used to prototype the LLM Agent that will act as the brain of the LLM-based Chatbot. Here I:
1. Define the tools available to the agent
2. Experiment with different prompt templates when constructing the agent

## Agent Construction

The API that the agent will use to find up to date anime related information is the [Jikan API](https://jikan.moe/). It parses the website MyAnimeList.net (MAL), one of the largest anime databases available, to satisfy the needs of the API.

The capabilities I need to provide the agent for core functionality are listed below:
- **Retrieve anime synopsis:** Get the anime's synopsis. *Wikipedia*
- **Retrieve anime ID:** Get the MAL ID of the anime. Scraped from the MAL site
- **Retrieve real-time statistics:** Get the live statistics on an anime. *Jikan API endpoint: getAnimeStatistics*
- **Retrieve recommendations:** Get other anime recommendations based on a given anime. *Jikan API endpoint: getAnimeRecommendations*
- **Retrieve sentiment:** Get sentiment around the anime. *Jikan API endpoint: getAnimeReviews*

### Imports

In [1]:
%pip install fuzzywuzzy
%pip install langchain
%pip install python-Levenshtein
%pip install openai
%pip install tiktoken
%pip install wikipedia

%pip install python-dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\halor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests, os, yaml, urllib.parse
from enum import Enum
from typing import Type, Optional
from fuzzywuzzy import fuzz
from pydantic import BaseModel, Field

from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.agents.agent_toolkits.openapi import planner
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_core.tools import ToolException
from langchain.requests import RequestsWrapper
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

### Retrieve Anime Synopsis

#### Tool

In [24]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wiki_tool = Tool.from_function(
        func=wikipedia.run,
        name="wikipedia_lookup",
        description="get anime synopsis",
        handle_tool_error=True
    )

wikipedia.run("Sousou no Frieren")

'Page: Frieren\nSummary: Frieren: Beyond Journey\'s End (Japanese: 葬送のフリーレン, Hepburn: Sōsō no Furīren, pp. "Frieren, the Final Farewell to the Dead") is a Japanese manga series written by Kanehito Yamada and illustrated by Tsukasa Abe. It has been serialized in Shogakukan\'s shōnen manga magazine Weekly Shōnen Sunday since April 2020, with its chapters collected in twelve tankōbon volumes as of December 2023. The series is licensed for English release in North America by Viz Media. An anime television series adaptation produced by Madhouse premiered in September 2023.\nBy December 2023, the manga had over 17 million copies in circulation. In 2021, Frieren: Beyond Journey\'s End won the 14th Manga Taishō and the New Creator Prize of the 25th annual Tezuka Osamu Cultural Prize.\n\nPage: Frieren (character)\nSummary: Frieren (Japanese: フリーレン, Hepburn: Furīren) is the title character and protagonist of the Japanese manga series Frieren: Beyond Journey\'s End, written by Kanehito Yamada and

#### Test

In [ ]:
tools = [wiki_tool]
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)
result = agent_executor({"input": "Tell me the plot of Jujutsu Kaisen"})

### Retrieve Anime ID

The agent needs the ability to retrieve the MyAnimeList (MAL) ID of a given anime as this ID must be included in the request for almost every endpoint of the Jikan API. Retrieving the MAL ID of a given anime is trivial as this information can simply be scraped from the internet, so the tool for this will simply enable the LLM to scrape the appropriate page of the site for the id using the name of the anime e.g. *getId(anime_name: string) -> int*.

#### Utils

In [4]:
def search_anime_name(animeName: str) -> dict[str, any]:
    url = 'https://api.jikan.moe/v4/anime?q={name}'.format(name=animeName)
    response = requests.get(url)
    response_json = response.json()
    return response_json['data']

def get_best_match(query: str, search_results: list[dict[str, any]]) -> tuple[str, str]:
    # Sort by levenshtein distance (edit distance) between query and title. The closest
    # match will be the final result
    search_results_sorted = sorted(search_results, key=lambda sr: fuzz.ratio(query, sr['title']))
    best_match = search_results_sorted[len(search_results) - 1]
    best_match_title = best_match['title']
    best_match_id = best_match['mal_id'] 
    return best_match_title, best_match_id

# Test
query = "Naruto Shippuden"
search_results = search_anime_name(query)
best_match = get_best_match(query, search_results)
print(best_match)

('Naruto: Shippuuden', 1735)


#### Tool

In [5]:
class AnimeIDLookupSchema(BaseModel):
    query: str = Field(description="anime name")


class AnimeIDLookupTool(BaseTool):
    name = "anime_id_lookup"
    description = "Useful for when you need to retrieve the id of an anime"
    args_schema: Type[AnimeIDLookupSchema] = AnimeIDLookupSchema

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            searchResults = search_anime_name(query)
            if searchResults:
                _, bestMatchId = get_best_match(query, searchResults)
                return bestMatchId
            raise ToolException("Anime not found")
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_id_lookup does not support async")

#### Test

Experimentation is carried out using OpenAI function calling, however useful resources for creating agents and learning how to use Llama 2 as an agent locally are listed below: 
- https://www.pinecone.io/learn/llama-2/
- https://philschmid.github.io/easyllm/examples/llama2-agent-example/#basic-example-of-using-a-tool-with-llama-2-70b
- https://medium.com/@gil.fernandes/langchain-chat-with-custom-tools-functions-and-memory-e34daa331aa7

In [5]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

tools = [anime_id_lookup_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)
result = agent_executor({"input": "Tell me the anime ID of Cowboy Bepop"}) # LLM should return 1
result = agent_executor({"input": "Tell me the anime ID of ..."}) # LLM should throw error

### Remaining functionality

The other aspects of the agent's functionality are non-trivial in that they all require allowing the agent to make API calls to the Jikan API. Langchain supports three main ways of doing this:
1. **OpenAPI** - If the API follows the OpenAPI spec, we can use the OpenAPI toolkit to parse the APIs OpenAPI specification and provide the LLM with the available tools directly. Refer to: https://python.langchain.com/docs/use_cases/apis and https://python.langchain.com/docs/integrations/toolkits/openapi
2. **Non-OpenAPI** - If the API doesn't follow the OpenAPI spec, we can define our own documentation for the API to construct the chain. As far as I know there is no standardised way of doing this, so it would require examining the examples in the Langchain docs and going through trial and error. Refer to: https://python.langchain.com/docs/use_cases/apis
3. **Custom Tools** - We define a custom tool for each endpoint required, and under the hood facilitate fetching the APIs response

Thankfully the JikanAPI provides an OpenAPI spec, so approaches 1 & 3 are viable. I will carry out experimentation below and decide which will be optimal for the final chatbot.

### OpenAPI

In [6]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)
requests_wrapper = RequestsWrapper()

# from urllib.request import urlretrieve
# urlretrieve("https://raw.githubusercontent.com/jikan-me/jikan-rest/master/storage/api-docs/api-docs.json","jikan_openapi.yaml")

with open("jikan_openapi.yaml", encoding="utf-8") as f:
    raw_jikan_api_spec = yaml.load(f, Loader=yaml.Loader)
jikan_api_spec = reduce_openapi_spec(raw_jikan_api_spec)

jikan_agent = planner.create_openapi_agent(jikan_api_spec, requests_wrapper, llm)

# Using simple OpenAPI agent
# NOTE: This agent does not have access to the id lookup tool, so the id must be included in the prompt

user_query = (
    "Tell me the plot of the anime Cowboy Bepop. It has an id of 1"
)
jikan_agent.run(user_query)

# OpenAPI agent + ID lookup tool
# Here the OpenAPI agent is used as a tool alongside the id lookup tool. Notice that the prompt no
# longer needs to include the anime ID. Instead, the main LLM is used as an orchestrator, and is instructed
# to pass the id retrieved using the id lookup tool to the OpenAPI agent.

jikan_tool = Tool.from_function(
        func=jikan_agent.run,
        name="anime_api_agent",
        description="useful for when you need to answer anime related queries. The input to this tool should be in the form '[query] anime id: [id]'",
        handle_tool_error=True
    )

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

tools = [anime_id_lookup_tool, jikan_tool]
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

result = agent_executor({"input": "Tell me the plot of the anime Cowboy Bepop"})

KeyboardInterrupt: 

### Custom Tools

#### Utils

In [54]:
# class JikanEndpoints(Enum):
#     GENERAL_INFO = "https://api.jikan.moe/v4/anime/{id}"
#     EPISODE_INFO = "https://api.jikan.moe/v4/anime/{id}/episodes"
#     CHARACTER_INFO = "https://api.jikan.moe/v4/anime/{id}/characters"

# class JikanResponseKeys(Enum):
#     TITLE = "TITLE"
#     URL = "URL"
#     TYPE = "TYPE"
#     STATUS = "STATUS"
#     NUM_EPISODES = "NUM_EPISODES"
#     EPISODES = "EPISODES"
#     CHARACTERS = "CHARACTERS"
#     AIRING = "AIRING"
#     AIRED = "AIRED"
#     DURATION = "DURATION"
#     RATING = "RATING"
#     SCORE = "SCORE"
#     SCORED_BY = "SCORED BY"
#     RANK = "RANK"
#     SYNOPSIS = "SYNOPSIS"
#     BACKGROUND = "BACKGROUND"
#     LICENSORS = "LICENSORS"
#     ANIMATION_STUDIOS = "ANIMATION STUDIOS [NOT A PRODUCER]"
#     PRODUCERS = "PRODUCERS"
#     GENRES = "GENRES"

# def jikan_get_request(endpoint: JikanEndpoints, animeId: int) -> dict[str, any]:
#     url = endpoint.value.format(id=animeId)
#     response = requests.get(url)
#     response_json = response.json()
#     data = response_json['data']
#     return data

# def construct_dict(keys: list[str], values: list[str]) -> dict[str, str]:
#     new_dict = dict(zip(keys, values))
#     return new_dict

# def parse_names(objList: list[dict[str, any]]) -> str:
#     namesList = list(map(lambda x: x['name'], objList))
#     names = ", ".join(namesList)
#     return names

# def parse_aired_dates(aired: dict[str, any]) -> str:
#     aired_string = 'from {start} to {end}' \
#                     .format(start=aired['from'], end=aired['to'])
#     return aired_string

# def parse_voice_actors(actors: list[dict[str, any]]) -> str:
#     va_info = list(map(lambda a: "Actor Name: {name}, Actor Language: {language}".format(name=a['person']['name'], language=a['language']), actors))
#     return va_info
    
# def parse_characters(characters: list[dict[str, any]]) -> str:
#     char_info = list(map(lambda c: "Name: {name}, Role: {role}, Voice Actors: {actors}" \
#                     .format(name=c['character']['name'], role=c['role'], actors=parse_voice_actors(c['voice_actors'])), characters))
#     return char_info

# def get_general_info(animeId: int) -> str:
#     endpoint = JikanEndpoints.GENERAL_INFO
#     data = jikan_get_request(endpoint, animeId)
    
#     keys = [
#         JikanResponseKeys.TITLE.value, 
#         JikanResponseKeys.URL.value, 
#         JikanResponseKeys.TYPE.value, 
#         JikanResponseKeys.STATUS.value,
#         JikanResponseKeys.NUM_EPISODES.value, 
#         JikanResponseKeys.AIRED.value, 
#         JikanResponseKeys.AIRING.value,
#         JikanResponseKeys.DURATION.value, 
#         JikanResponseKeys.RATING.value, 
#         JikanResponseKeys.SCORE.value, 
#         JikanResponseKeys.SCORED_BY.value,
#         JikanResponseKeys.RANK.value, 
#         JikanResponseKeys.SYNOPSIS.value, 
#         JikanResponseKeys.BACKGROUND.value, 
#         JikanResponseKeys.LICENSORS.value,
#         JikanResponseKeys.ANIMATION_STUDIOS.value, 
#         JikanResponseKeys.PRODUCERS.value, 
#         JikanResponseKeys.GENRES.value
#         ]
#     values = [
#         data['title'], 
#         data['url'], 
#         data['type'], 
#         data['status'], 
#         data['episodes'],
#         data['airing'],
#         parse_aired_dates(data['aired']),
#         data['duration'],
#         data['rating'],
#         data['score'],
#         data['scored_by'],
#         data['rank'],
#         data['synopsis'],
#         data['background'],
#         parse_names(data['licensors']),
#         parse_names(data['studios']),
#         parse_names(data['producers']),
#         parse_names(data['genres'])
#         ]
    
#     data_parsed = construct_dict(keys, values)
#     data_as_string = str(data_parsed)
#     return data_as_string

# def get_character_info(animeId: int) -> str:
#     url = JikanEndpoints.CHARACTER_INFO
#     data = jikan_get_request(url, animeId)
    
#     keys = [
#         JikanResponseKeys.CHARACTERS.value
#         ]
#     values = [
#         parse_characters(data)
#         ]
    
#     data_parsed = construct_dict(keys, values)
#     data_as_string = str(data_parsed)
#     return data_as_string

# def get_anime_info(animeId: int, option: str) -> str:
#     if (option == "general_info"):
#         return get_general_info(animeId)
#     elif (option == "character_info"):
#         return get_character_info(animeId)

# option = "character_info"
# print(len(get_anime_info(20, option)))

86330


In [6]:
class JikanEndpoints(Enum):
    STATISTICS = "https://api.jikan.moe/v4/anime/{id}/statistics"
    RECOMMENDATIONS = "https://api.jikan.moe/v4/anime/{id}/recommendations"
    REVIEWS = "https://api.jikan.moe/v4/anime/{id}/reviews"
    
def jikan_get_request(endpoint: JikanEndpoints, anime_id: int, params: Optional[dict[str, str]] = None) -> dict[str, any]:
    url = endpoint.value.format(id=anime_id)
    
    if params:
        params_formatted = urllib.parse.urlencode(params)
        url_with_params = "{url}?{params}".format(url=url, params=params_formatted)
        url = url_with_params
    
    response = requests.get(url)
    response_json = response.json()
    data = response_json['data']
    return data

def format_scores(scores: list[dict[str, any]]) -> str:
    scores_formatted = list(map(lambda s: "{}/10, Votes for score: {}, Percentage of total: {}".format(s['score'], s['votes'], s['percentage']), scores))
    scores_formatted = "\n".join(scores_formatted)
    return scores_formatted

def format_recommendations(recommendations: list[dict[str, any]]) -> str:
    recommendations_formatted = list(map(lambda s: "{}".format(s['entry']['title']), recommendations))
    recommendations_formatted = ", ".join(recommendations_formatted)
    return recommendations_formatted

def filter_reviews(raw_reviews: list[dict[str, any]], top_k: int) -> str:
    reviews_filtered = sorted(raw_reviews, key=lambda r: r['reactions']['overall'])
    return reviews_filtered[len(reviews_filtered) - top_k:]

def summarise_review(review: str) -> str:
    key: str = os.getenv("OPENAI_API_KEY")
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)
    template = """Summarise this review in ONLY one sentence:
    Review: {review}"""
    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    result = llm_chain.run(review)
    return result

def format_reviews(reviews: list[dict[str, any]]) -> str:
    reviews_formatted = list(map(lambda r: "Review Summary: {}".format(summarise_review(r['review'])), reviews))
    reviews_formatted = "\n".join(reviews_formatted)
    return reviews_formatted
    
def get_anime_statistics(anime_id: int) -> str:
    endpoint = JikanEndpoints.STATISTICS
    data = jikan_get_request(endpoint, anime_id)

    stats_formatter = "SCORES: {}\nSource: MyAnimeList.net"
    scores_as_string = format_scores(data['scores'])
    data_parsed = stats_formatter.format(scores_as_string)
    return data_parsed

def get_anime_recommendations(anime_id: int, num_recommendations: int = 5) -> str:
    endpoint = JikanEndpoints.RECOMMENDATIONS
    data = jikan_get_request(endpoint, anime_id)
    data_filtered = data[:num_recommendations]
    
    recommendations_formatter = "RECOMMENDATIONS: {}\nSource: MyAnimeList.net"
    recommendations_as_string = format_recommendations(data_filtered)
    data_parsed = recommendations_formatter.format(recommendations_as_string)
    return data_parsed

def get_anime_reviews(anime_id: int, top_k: int = 5) -> str:
    endpoint = JikanEndpoints.REVIEWS
    params = { 'preliminary' : 'true' }
    data = jikan_get_request(endpoint, anime_id, params)
    data_filtered = filter_reviews(data, top_k)
    
    reviews_formatter = "REVIEWS: {}\nSource: MyAnimeList.net"
    reviews_as_string = format_reviews(data_filtered)
    data_parsed = reviews_formatter.format(reviews_as_string)
    return data_parsed

def get_anime_info(anime_id: int, option: str) -> str:
    if (option == "statistics"):
        return get_anime_statistics(anime_id)
    elif (option == "recommendations"):
        return get_anime_recommendations(anime_id)
    elif (option == "reviews"):
        return get_anime_reviews(anime_id)

option = "reviews"
print(get_anime_info(20, option))

REVIEWS: Review Summary: This review criticizes the series for its lack of interesting plot, excessive filler episodes, and unappealing characters, and advises against watching it due to the wasted time and lack of worth.
Review Summary: The reviewer enjoyed Naruto and found the characters and supporting cast to be well-developed, but acknowledged pacing issues and the potential for frustration with certain arcs.
Review Summary: The reviewer praises Naruto for its unique ninja aspect, character development, emotional depth, and exploration of themes such as friendship and bonds, while acknowledging that the series has changed over time and may not appeal to everyone.
Review Summary: The reviewer finds Naruto to be a popular anime series that appeals to young teenagers, but they personally found it to be average and have seen better shows, although they appreciate the soundtrack.
Review Summary: The reviewer finds Naruto to be overhyped and disappointing, with too many fights, bad fille

#### Tools

In [7]:
class AnimeStatsInputSchema(BaseModel):
    id: int = Field(description="anime id")

class AnimeStatsTool(BaseTool):
    name = "anime_stats"
    description = "Useful for looking up the statistics on an anime, like the ratings people have given it"
    args_schema: Type[AnimeStatsInputSchema] = AnimeStatsInputSchema

    def _run(
        self,
        id: int,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            anime_stats = get_anime_statistics(id)
            return anime_stats
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        id: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_stats does not support async")

class AnimeRecommendationsInputSchema(BaseModel):
    id: int = Field(description="anime id")
    num: int = Field(description="number of recommendations")

class AnimeRecommendationsTool(BaseTool):
    name = "anime_recommendations"
    description = "Useful for looking up recommendations based on a given anime"
    args_schema: Type[AnimeRecommendationsInputSchema] = AnimeRecommendationsInputSchema

    def _run(
        self,
        id: int,
        num: int,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            anime_recommendations = get_anime_recommendations(id, num)
            return anime_recommendations
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        id: str,
        num:int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_stats does not support async")
    
class AnimeReviewsInputSchema(BaseModel):
    id: int = Field(description="anime id")

class AnimeReviewsTool(BaseTool):
    name = "anime_reviews"
    description = "Useful for finding reviews for an anime and get an idea of sentiment"
    args_schema: Type[AnimeReviewsInputSchema] = AnimeReviewsInputSchema

    def _run(
        self,
        id: int,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        try:    
            anime_reviews = get_anime_reviews(id)
            return anime_reviews
        except Exception as e:
            raise ToolException(e)

    async def _arun(
        self,
        id: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("anime_stats does not support async")

#### Test

In [25]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

anime_stats_tool = AnimeStatsTool()
anime_stats_tool.handle_tool_error = True

anime_recommendations_tool = AnimeRecommendationsTool()
anime_recommendations_tool.handle_tool_error = True

anime_reviews_tool = AnimeReviewsTool()
anime_reviews_tool.handle_tool_error = True

tools = [anime_id_lookup_tool, anime_stats_tool, anime_recommendations_tool, anime_reviews_tool]
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, agent_kwargs=agent_kwargs, memory=memory
)

result =agent_executor({"input": "Do you think I should watch Jujutsu Kaisen?"})



> Entering new AgentExecutor chain...

Invoking: `anime_id_lookup` with `{'query': 'Sousou no Frieren'}`


52991
Invoking: `anime_stats` with `{'id': 52991}`


SCORES: 1/10, Votes for score: 2259, Percentage of total: 1.9
2/10, Votes for score: 189, Percentage of total: 0.2
3/10, Votes for score: 193, Percentage of total: 0.2
4/10, Votes for score: 320, Percentage of total: 0.3
5/10, Votes for score: 798, Percentage of total: 0.7
6/10, Votes for score: 1460, Percentage of total: 1.2
7/10, Votes for score: 4390, Percentage of total: 3.6
8/10, Votes for score: 13920, Percentage of total: 11.5
9/10, Votes for score: 30865, Percentage of total: 25.6
10/10, Votes for score: 66274, Percentage of total: 54.9
Source: MyAnimeList.net"Sousou no Frieren" is an anime series that is based on a manga of the same name. It follows the story of Frieren, a 10,000-year-old witch who has been granted eternal youth and immortality. Frieren has lived through countless generations and has witnessed the ris

### Conclusion

Using OpenAPI has a major drawback in that it naturally causes the LLM to perform inference more than necessary due to the sheer number of endpoints available. As the LLM performs reasoning to respond to a user query, it calls any endpoint that may be helpful, despite the fact that often only one or two endpoints will return the necessary information. Considering that only a subset of the endpoints is necessary to achieve core functionality, this means the cost of inference is unnecessarily high. Thus, for AniMate, I will use the custom tools approach, and only implement tools for a subset of the endpoints. This will keep the number of tokens required for inference down and reduce the complexity of AniMate's implementation.

## Final Agent



In [26]:
key: str = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, openai_api_key=key)

anime_id_lookup_tool = AnimeIDLookupTool()
anime_id_lookup_tool.handle_tool_error = True

anime_stats_tool = AnimeStatsTool()
anime_stats_tool.handle_tool_error = True

anime_recommendations_tool = AnimeRecommendationsTool()
anime_recommendations_tool.handle_tool_error = True

anime_reviews_tool = AnimeReviewsTool()
anime_reviews_tool.handle_tool_error = True

tools = [wiki_tool, anime_id_lookup_tool, anime_stats_tool, anime_recommendations_tool, anime_reviews_tool]
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, agent_kwargs=agent_kwargs, memory=memory
)

result = agent_executor({"input": "Do you think I should watch Sousou no Frieren?"})
result = agent_executor({"input": "Ok, what are people saying about Sousou no Frieren?"})
result = agent_executor({"input": "Interesting, what's it about?"})



> Entering new AgentExecutor chain...

Invoking: `anime_id_lookup` with `{'query': 'Sousou no Frieren'}`


52991
Invoking: `anime_stats` with `{'id': 52991}`


SCORES: 1/10, Votes for score: 2259, Percentage of total: 1.9
2/10, Votes for score: 189, Percentage of total: 0.2
3/10, Votes for score: 193, Percentage of total: 0.2
4/10, Votes for score: 320, Percentage of total: 0.3
5/10, Votes for score: 798, Percentage of total: 0.7
6/10, Votes for score: 1460, Percentage of total: 1.2
7/10, Votes for score: 4390, Percentage of total: 3.6
8/10, Votes for score: 13920, Percentage of total: 11.5
9/10, Votes for score: 30865, Percentage of total: 25.6
10/10, Votes for score: 66274, Percentage of total: 54.9
Source: MyAnimeList.netBased on the statistics from MyAnimeList, Sousou no Frieren has a high average score of 9.0/10 with a significant number of votes. This suggests that the majority of viewers have enjoyed the anime and have given it positive ratings. Therefore, it seems like Sousou